# Script to Run the Proposed Inventory Model Simulation

## Import

In [4]:
import math
import pandas as pd
import numpy as np
from scipy import stats

## Initialize Parameters 

In [5]:
def get_demand_statistics(data):
    return data.mean(), data.std()

def calculate_rq_parameters(params, demand_data):
    # Unpack parameters
    L = params['L']  
    m = params['m']  
    K = params['K']  
    h = params['h']  
    c = params['c']  
    w = params['w'] 
    p = params['p']  
    
    # Get demand statistics
    mean_demand, std_demand = get_demand_statistics(demand_data)
    
    # Determine base order quantity (Q_base)
    # Modified EOQ formula considering outdating cost
    annual_demand = mean_demand * 365
    EOQ = math.sqrt((2 * K * annual_demand) / (h + w * (1/m)))
    Q_base = min(EOQ, mean_demand * m)
    
    # Determine base reorder point (r_base)
    # Demand during lead time
    DL = mean_demand * L
    
    # Initial safety factor (can be adjusted)
    z = stats.norm.ppf(0.95)
    
    # Safety stock
    SS = z * std_demand * math.sqrt(L)
    
    # Base reorder point
    r_base = DL + SS
    
    # Adjust for perishability
    max_inv = mean_demand * m
    if r_base + Q_base > max_inv:
        Q_base = max_inv - r_base
    
    # Expected shortage per cycle
    E_shortage = std_demand * math.sqrt(L) * (stats.norm.pdf(z) - z * (1 - stats.norm.cdf(z)))
    
    # Adjust safety factor to balance costs
    optimal_z = stats.norm.ppf(p / (p + h + w))
    
    # Recalculate safety stock and reorder point
    SS = optimal_z * std_demand * math.sqrt(L)
    r_base = DL + SS
    
    return {
        'Q_base': Q_base,
        'r_base': r_base,
        'EOQ': EOQ,
        'safety_stock': SS,
        'optimal_service_level': stats.norm.cdf(optimal_z),
        'mean_demand': mean_demand,
        'std_demand': std_demand
    }

df = pd.read_csv('RestaurantDataVets_All_2to5.csv')

df['2to5'] = pd.to_numeric(df['2to5'], errors='coerce')

# Remove any rows with NaN values in the '2to5' column
df = df.dropna(subset=['2to5'])

# Scale the demand data
scale_factor = 1027.92  # Adjust this value as needed
df['2to5_scaled'] = df['2to5'] / scale_factor


params = {
    'L': 1,
    'm': 3,
    'K': 100,
    'h': 1,
    'c': 5,
    'w': 5,
    'p': 20
}

results = calculate_rq_parameters(params, df['2to5_scaled'])
print("Results (scaled):")
print(results)

# Analyze demand patterns
print("\nDemand Analysis (unscaled):")
print(f"Total days of data: {len(df)}")
print(f"Days with zero demand: {(df['2to5'] == 0).sum()}")
print(f"Maximum daily demand: {df['2to5'].max()}")
print(f"Minimum non-zero daily demand: {df['2to5'][df['2to5'] > 0].min()}")
print(f"Average daily demand: {df['2to5'].mean():.2f}")
print(f"Standard deviation of daily demand: {df['2to5'].std():.2f}")

Results (scaled):
{'Q_base': 1.2444392868481442, 'r_base': 1.3382320018873868, 'EOQ': 165.4542064879762, 'safety_stock': 0.3382285518575836, 'optimal_service_level': 0.7692307692307694, 'mean_demand': 1.0000034500298032, 'std_demand': 0.45935249242317755}

Demand Analysis (unscaled):
Total days of data: 1111
Days with zero demand: 15
Maximum daily demand: 2889.23
Minimum non-zero daily demand: 225.3
Average daily demand: 1027.92
Standard deviation of daily demand: 472.18
